In [ ]:
data_list=[]
import json

with open(rf"./saves/Qwen2-VL-7B-Instruct/lora/sft/generated_predictions.jsonl",'r',encoding='utf-8') as f:
        all_=f.readlines()
        for i in all_:
             data_list.append(json.loads(i))
                


In [ ]:
data_list

In [4]:
def get_label_type(msg):
    if '非瘤变' in msg:
        return '非瘤变'
    elif '溃疡' in msg or '瘤变' in msg:
        return '高风险'
    elif 'SMT' in msg:
        return 'SMT'
    elif '黄斑瘤' in msg:
        return '黄斑瘤'
    elif '息肉' in msg:
        return '息肉'
    else:
        return '其他'
    
def get_label_dict(msg):
    #print(msg)
    label=get_label_type(msg)

    if  '溃疡' in msg:
        label='溃疡'
    label_info={"形态":'',"颜色":'',"表面":'',"边界":'',"出血":'无'}
    shape_list=['浅凹陷','深凹陷','半球形','平滑型','梭状','半环状','不规则']
    lesion_dict={"溃疡":
                 {
                     "形态":['浅凹陷','深凹陷'],
                     "颜色":['苔'],
                     "表面":[],
                     "边界":['形状规则','形状不规则'],
                     "出血":['未见出血','不伴出血','无自发性出血','出血'],
                 },
                 "SMT":
                 {
                     "形态":['半球形','椭球形','球形','平滑型','梭状','半环状','环状','不规则'],
                     "颜色":['同色调'],
                     "表面":['光滑','伴糜烂','伴充血糜烂','脐状凹陷'],
                     "边界":['边界清晰','边界不清晰','未见清晰边界'],
                     "出血":['未见出血','不伴出血','无自发性出血','出血'],
                 },
                 "黄斑瘤":
                 {
                     "形态":['隆起','扁平'],
                     "颜色":['黄色','黄白色'],
                     "表面":['粗糙'],
                     "边界":['清晰'],
                     "出血":['未见出血','不伴出血','无自发性出血','出血'],
                 },
                 "息肉":
                 {
                     "形态":['半球形隆起','隆起','扁平'],
                     "颜色":['同色调','发红'],
                     "表面":['光滑','粗糙'],
                     "边界":['清晰'],
                     "出血":['未见出血','不伴出血','无自发性出血','出血'],
                 },
                 "非瘤变":
                 {
                     "形态":['隆起型病变，中央凹陷','凹陷型病变，周边隆起','隆起','凹陷','平坦',],
                     "颜色":['发红','同色调','褪色调'],
                     "表面":['糜烂','光滑','粗糙'],
                     "边界":['未见清晰边界','边界不清晰','边界清晰','边界形状规则','边界形状不规则'],
                     "出血":['未见出血','不伴出血','无自发性出血','出血'],
                 },
                 "瘤变":
                 {
                     "形态":['隆起型病变，中央凹陷','凹陷型病变，周边隆起','隆起','凹陷','平坦','不规则结节状隆起','略微隆起'],
                     "颜色":['发红','同色调','褪色调','色调不均一'],
                     "表面":['粗糙','黏膜呈断崖状','胃体黏膜皱襞中断'],
                     "边界":['未见清晰边界','边界不清晰','边界清晰','边界形状规则','边界形状不规则','边界形状不规则呈毛刺状'],
                     "出血":['未见出血','不伴出血','无自发性出血','出血'],
                 },
                 
                }
    for k,v in lesion_dict.get(label,{}).items():
        for i in v:
            if i in msg:
                label_info[k]=i
                break

    if label_info['边界']=='未见清晰边界':
        label_info['边界']='不清晰'
    if label_info['边界']=='边界不清晰':
        label_info['边界']='不清晰'
    if label_info['边界']=='边界清晰':
        label_info['边界']='清晰'
    if '未见' in label_info['出血'] or '不伴' in label_info['出血'] or '无' in label_info['出血']:
        label_info['出血']='无'
    else:
        label_info['出血']='有'
    print(label_info)
    return label_info

def cal_score(label_dict_true,label_dict_predict):
    total=len(label_dict_true.keys())
    yes=0
    for k,v in label_dict_true.items():
        if v == label_dict_predict.get(k,"") or v=='':
            yes+=1
    total_score=yes/total if total else 0
    print('总：',total_score) 
    return total_score

def cal(y_true,y_pred):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {accuracy}")

    precision = precision_score(y_true, y_pred, average='macro')
    print(f"Precision (macro): {precision}")

    recall = recall_score(y_true, y_pred, average='macro')
    print(f"Recall (macro): {recall}")

    f1 = f1_score(y_true, y_pred, average='macro')
    print(f"F1-score (macro): {f1}")

    conf_matrix = confusion_matrix(y_true, y_pred)
    print(f"Confusion Matrix:\n{conf_matrix}")

In [ ]:
lesion_cn_to_en={ "高风险":"High-risk lesion","非瘤变":"Erosion","息肉":"Benign polyp","SMT":"Subepithelial Lesion","黄斑瘤":"Xanthelasma","其他":"other"}
single_dict={}
error_data=[]
class_labels=[]
true_list=[]
pre_list=[]

error_list=[]
score_dict={}
for i in data_list:
    label=i['label']
    predict=i['predict']
    
    label_dict_true=lesion_cn_to_en[get_label_type(label)]
    if label_dict_true!='其他':
        label_dict_predict=lesion_cn_to_en[get_label_type(predict)]
        true_list.append(label_dict_true)
        pre_list.append(label_dict_predict)
        if label_dict_true not in class_labels:
            class_labels.append(label_dict_true)
        if label_dict_predict not in class_labels:
            class_labels.append(label_dict_predict)

        if label_dict_true in single_dict:
            single_dict[label_dict_true][0]+=1
            single_dict[label_dict_true][1]+=1 if label_dict_true==label_dict_predict else 0
        else:
            single_dict[label_dict_true]=[0,0]
            single_dict[label_dict_true][0]=1
            single_dict[label_dict_true][1]=1 if label_dict_true==label_dict_predict else 0
        print(label)
        label_dict_true=get_label_dict(label)
        lesion=lesion_cn_to_en[get_label_type(label)]
        print(predict)
        label_dict_predict=get_label_dict(predict)
        total_score=cal_score(label_dict_true,label_dict_predict)
        shape_score=1 if label_dict_true.get("形态")==label_dict_predict.get("形态") or label_dict_true.get("形态")==''  else 0
        color_score=1 if label_dict_true.get("颜色")==label_dict_predict.get("颜色") or label_dict_true.get("颜色")=='' else 0
        surface_score=1 if label_dict_true.get("表面")==label_dict_predict.get("表面") or label_dict_true.get("表面")=='' else 0
        boundary_score=1 if label_dict_true.get("边界")==label_dict_predict.get("边界") or label_dict_true.get("边界")=='' else 0
        blod_score=1 if label_dict_true.get("出血")==label_dict_predict.get("出血") or label_dict_true.get("出血")=='' else 0
        if lesion not in score_dict:
            score_dict[lesion]={"形态":shape_score,
                                "颜色":color_score,
                                "表面":surface_score,
                                "边界":boundary_score,
                                "出血":blod_score,
                               '总':total_score,
                                '总_score':shape_score+color_score+surface_score+boundary_score+blod_score,
                                'count':1}
        else:
            score_dict[lesion]['形态']+=shape_score
            score_dict[lesion]['颜色']+=color_score
            score_dict[lesion]['表面']+=surface_score
            score_dict[lesion]['边界']+=boundary_score
            score_dict[lesion]['出血']+=blod_score
            score_dict[lesion]['总']+=total_score
            score_dict[lesion]['总_score']+=shape_score+color_score+surface_score+boundary_score+blod_score
            score_dict[lesion]['count']+=1

In [ ]:
single_dict

In [ ]:
total=0
total_count=0
total_=0
total_count_=0
for k,v in single_dict.items():
    if v[0]>0:
        print(f"{k}:accuracy：{round(v[1]/v[0]*100,2)}%;count：{v[1]};nums：{v[0]}")
        total+=v[1]/v[0]
        total_count+=1
        total_+=v[1]
        total_count_+=v[0]
print(f"toal:{round(total/total_count*100,2)}%;toal_:{round(total_/total_count_*100,2)}%;count：{total_};nums：{total_count_}")

In [ ]:
cal(true_list,pre_list)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(true_list,pre_list, labels=class_labels)

fig, ax = plt.subplots()

cax = ax.matshow(cm, cmap='Blues')

fig.colorbar(cax)


ax.set_xticklabels([''] + class_labels, rotation=45, ha='left')
ax.set_yticklabels([''] + class_labels)

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, cm[i, j], va='center', ha='center')

ax.set_xlabel('Predict')
ax.set_ylabel('True')

In [ ]:
score_dict

In [ ]:
type_cn_to_en={ "形态":"Morphology","颜色":"Tone of color","表面":"Roughness","边界":"Boundary","出血":"Hemorrhage","总":"overall","其他":"other"}
type_score={'总':{"score":0,"count":0}}
for k,v in score_dict.items():
    print(f"Lesion type:{k}")
    total_score=v['总_score']
    for k1,v1 in v.items():
        
        if k1 not in ['count','总_score']:
            if k1=='总':
                print(f"Dimension:{type_cn_to_en[k1]},score:{total_score},count:{v['count']*5},accuracy:{round(v1/v['count']*100,2)}%")
            else:
                print(f"Dimension:{type_cn_to_en[k1]},score:{v1},count:{v['count']},accuracy:{round(v1/v['count']*100,2)}%")
                if k1 in type_score:
                    #type_score[k1]+=v1/v['count']
                    type_score[k1]['score']+=v1
                    type_score[k1]['count']+=v['count']
                else:
                    #type_score[k1]=v1/v['count']
                    type_score[k1]={"score":v1,"count":v['count']}
    type_score['总']['score']+=total_score
    type_score['总']['count']+=v['count']*5

In [ ]:
type_score

In [ ]:
for k,v in type_score.items():
    print(f"Dimension：{type_cn_to_en[k]},score:{v['score']},count:{v['count']},accuracy：{round(v['score']/v['count']*100,2)}%")


In [ ]:
# pip install jieba

In [ ]:
# pip install rouge_chinese

In [19]:
from dataclasses import dataclass
from typing import TYPE_CHECKING, Dict, Sequence, Tuple, Union

import numpy as np
import jieba  # type: ignore


from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

from rouge_chinese import Rouge


def ComputeMetrics( preds,labels) -> Dict[str, float]:
        r"""
        Uses the model predictions to compute metrics.
        """

        score_dict = {"rouge-1": [], "rouge-2": [], "rouge-l": [], "bleu-4": []}

        for pred, label in zip(preds, labels):
            hypothesis = list(jieba.cut(pred))
            reference = list(jieba.cut(label))

            if len(" ".join(hypothesis).split()) == 0 or len(" ".join(reference).split()) == 0:
                result = {"rouge-1": {"f": 0.0}, "rouge-2": {"f": 0.0}, "rouge-l": {"f": 0.0}}
            else:
                rouge = Rouge()
                scores = rouge.get_scores(" ".join(hypothesis), " ".join(reference))
                result = scores[0]

            for k, v in result.items():
                score_dict[k].append(round(v["f"] * 100, 4))

            bleu_score = sentence_bleu([list(label)], list(pred), smoothing_function=SmoothingFunction().method3)
            score_dict["bleu-4"].append(round(bleu_score * 100, 4))

        return {k: float(np.mean(v)) for k, v in score_dict.items()}

In [ ]:
preds=[i.get('predict') for i in data_list]
labels=[i.get('label') for i in data_list]
ComputeMetrics(preds,labels)